In [1]:
import meshio
import numpy as np
import pandas as pd
from scipy import ndimage

# Reading global mesh information

In [14]:
dir_lvl = 'lvl_8'

In [15]:
mesh_struct = np.load(dir_lvl+'/mesh.npz')

print("Field names: {}".format(mesh_struct.files))

ngbIDs = mesh_struct['n']
vertices = mesh_struct['v']
cells = mesh_struct['c']

Field names: ['c', 'n', 'v']


In [16]:
mesh_lonlat = np.load(dir_lvl+'/icoord_elev.npz')

print("Field names: {}".format(mesh_lonlat.files))

coords = mesh_lonlat['v']

Field names: ['v']


## Reading paleo-displacement grids

In [17]:
def buildPaleoDisp(gPlates=None, outfile=None, visvtk=False):
    '''
    Reading paleo-displacement grids
    '''

    # Open gPlates 1 degree 3D displacement maps (xy files)
    data = pd.read_csv(gPlates, sep=r'\s+', engine='c', header=None, skiprows=[0,1,2,3,4,5,65166],
                       error_bad_lines=True, na_filter=False, dtype=np.float, low_memory=False)

    # Read dataset
    lon = data.values[:,0]
    lat = data.values[:,1]

    # Conversion from cm/yr to m/yr
    tmpx = data.values[:,2]/100.
    tmpy = data.values[:,3]/100.
    tmpz = data.values[:,4]/100.

    # Reshape dataset
    tmpx = np.fliplr(tmpx.reshape((181,360)).T)
    tmpy = np.fliplr(tmpy.reshape((181,360)).T)
    tmpz = np.fliplr(tmpz.reshape((181,360)).T)
    dispX = np.zeros((361,181))
    dispY = np.zeros((361,181))
    dispZ = np.zeros((361,181))
    dispX[:360,:] = tmpx
    dispX[-1,:] = tmpx[0,:]
    dispY[:360,:] = tmpy
    dispY[-1,:] = tmpy[0,:]
    dispZ[:360,:] = tmpz
    dispZ[-1,:] = tmpz[0,:]

    # Interpolate the paleo displacement on global mesh
    dX = ndimage.map_coordinates(dispX , coords/10., order=3, mode='nearest').astype(np.float)
    dY = ndimage.map_coordinates(dispY , coords/10., order=3, mode='nearest').astype(np.float)
    dZ = ndimage.map_coordinates(dispZ , coords/10., order=3, mode='nearest').astype(np.float)
    disps = np.stack((dX, dY, dZ)).T

    # Save the mesh as compressed numpy file for global simulation
    np.savez_compressed(outfile, xyz=disps)

    if visvtk:
        vis_mesh = meshio.Mesh(vertices, {'triangle': cells}, point_data={"ux":dX,"uy":dY,"uz":dZ})
        meshio.write(outfile+".vtk", vis_mesh)
        print("Writing VTK file {}".format(outfile+".vtk"))

    print("Processing {} to create {}".format(gPlates,outfile+".npz"))

    return

In [18]:
# Specify most recent time in Ma
startMa = 0
# Specify deepest time in Ma
endMa = 10
# Specify paleodisp interval
dtMa = 1

In [ ]:
timeframe = np.flipud(np.arange(startMa,endMa+dtMa,dtMa))

for k in range(len(timeframe)):
    f_gplates = 'data/gPlates-velocity/velocity_'+str(k)+'.00Ma.xy'
    paleo_disp = dir_lvl+'/disp'+str(k)+'Ma'
    if k == 0:
        buildPaleoDisp(gPlates=f_gplates, outfile=paleo_disp, visvtk=True) 
    else:
        buildPaleoDisp(gPlates=f_gplates, outfile=paleo_disp, visvtk=False) 

Writing VTK file lvl_8/disp0Ma.vtk
Processing data/gPlates-velocity/velocity_0.00Ma.xy to create lvl_8/disp0Ma.npz
Processing data/gPlates-velocity/velocity_1.00Ma.xy to create lvl_8/disp1Ma.npz
Processing data/gPlates-velocity/velocity_2.00Ma.xy to create lvl_8/disp2Ma.npz
Processing data/gPlates-velocity/velocity_3.00Ma.xy to create lvl_8/disp3Ma.npz
Processing data/gPlates-velocity/velocity_4.00Ma.xy to create lvl_8/disp4Ma.npz
Processing data/gPlates-velocity/velocity_5.00Ma.xy to create lvl_8/disp5Ma.npz
Processing data/gPlates-velocity/velocity_6.00Ma.xy to create lvl_8/disp6Ma.npz
Processing data/gPlates-velocity/velocity_7.00Ma.xy to create lvl_8/disp7Ma.npz
